<a href="https://colab.research.google.com/github/DiegoCorona/BEDU-Proyecto-Final-Estudio-de-mercado-de-Apps-M-viles-y-predicci-n-de-xito-/blob/main/Estudio_de_mercado_de_Apps_M%C3%B3viles_y_predicci%C3%B3n_de_exito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Dataset Proyecto/Bedu_Proyecto_Modulo_4/Dataset_limpio.csv', index_col=0)
data.head(1)

,App,Categoria,Reviews,Tamanio_MB,Instalaciones_minimas_estimadas,App_gratuita,App_pago,Precio,Rating_del_contenido,Ultimo_updated,Anio_ultimo_updated,Version_actual,Version_Android,Rating,Exito_app
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,159,19.0,10000,1,0,0.0,Everyone,"January 7, 2018",2018,1.0.0,4.0.3 and up,4.1,1


In [ ]:
expanded = pd.get_dummies(data = data, columns = ['Categoria'])
expanded = expanded.drop(columns = ['App', 'Rating_del_contenido', 'Ultimo_updated', 'Anio_ultimo_updated','Version_actual', 'Version_Android', 'Rating'])

In [ ]:
expanded.head()

,Reviews,Tamanio_MB,Instalaciones_minimas_estimadas,App_gratuita,App_pago,Precio,Exito_app,Categoria_ART_AND_DESIGN,Categoria_AUTO_AND_VEHICLES,Categoria_BEAUTY,Categoria_BOOKS_AND_REFERENCE,Categoria_BUSINESS,Categoria_COMICS,Categoria_COMMUNICATION,Categoria_DATING,Categoria_EDUCATION,Categoria_ENTERTAINMENT,Categoria_EVENTS,Categoria_FAMILY,Categoria_FINANCE,Categoria_FOOD_AND_DRINK,Categoria_GAME,Categoria_HEALTH_AND_FITNESS,Categoria_HOUSE_AND_HOME,Categoria_LIBRARIES_AND_DEMO,Categoria_LIFESTYLE,Categoria_MAPS_AND_NAVIGATION,Categoria_MEDICAL,Categoria_NEWS_AND_MAGAZINES,Categoria_PARENTING,Categoria_PERSONALIZATION,Categoria_PHOTOGRAPHY,Categoria_PRODUCTIVITY,Categoria_SHOPPING,Categoria_SOCIAL,Categoria_SPORTS,Categoria_TOOLS,Categoria_TRAVEL_AND_LOCAL,Categoria_VIDEO_PLAYERS,Categoria_WEATHER
0,159,19.0,10000,1,0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,967,14.0,500000,1,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,87510,8.7,5000000,1,0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,215644,25.0,50000000,1,0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,967,2.8,100000,1,0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = expanded.drop(columns= ['Exito_app'])
y = expanded['Exito_app']

### Normalizando algunas variables.

In [ ]:
X_norm = X.copy()
X_norm['Reviews'] = (X['Reviews'] - X['Reviews'].mean())/X['Reviews'].std()
X_norm['Instalaciones_minimas_estimadas'] = (X['Instalaciones_minimas_estimadas'] - X['Instalaciones_minimas_estimadas'].mean())/X['Instalaciones_minimas_estimadas'].std()
X_norm['Tamanio_MB'] = (X['Tamanio_MB'] - X['Tamanio_MB'].mean())/X['Tamanio_MB'].std()
X_norm['Precio'] = (X['Precio'])/(X['Precio'].max()-X['Precio'].min())

In [ ]:
X_norm.head()

,Reviews,Tamanio_MB,Instalaciones_minimas_estimadas,App_gratuita,App_pago,Precio,Categoria_ART_AND_DESIGN,Categoria_AUTO_AND_VEHICLES,Categoria_BEAUTY,Categoria_BOOKS_AND_REFERENCE,Categoria_BUSINESS,Categoria_COMICS,Categoria_COMMUNICATION,Categoria_DATING,Categoria_EDUCATION,Categoria_ENTERTAINMENT,Categoria_EVENTS,Categoria_FAMILY,Categoria_FINANCE,Categoria_FOOD_AND_DRINK,Categoria_GAME,Categoria_HEALTH_AND_FITNESS,Categoria_HOUSE_AND_HOME,Categoria_LIBRARIES_AND_DEMO,Categoria_LIFESTYLE,Categoria_MAPS_AND_NAVIGATION,Categoria_MEDICAL,Categoria_NEWS_AND_MAGAZINES,Categoria_PARENTING,Categoria_PERSONALIZATION,Categoria_PHOTOGRAPHY,Categoria_PRODUCTIVITY,Categoria_SHOPPING,Categoria_SOCIAL,Categoria_SPORTS,Categoria_TOOLS,Categoria_TRAVEL_AND_LOCAL,Categoria_VIDEO_PLAYERS,Categoria_WEATHER
0,-0.163449,-0.108131,-0.196052,1,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.163192,-0.336461,-0.190682,1,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.135666,-0.578490,-0.141360,1,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.094912,0.165864,0.351854,1,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-0.163192,-0.847918,-0.195066,1,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, 
                                                    test_size = 0.2, 
                                                    random_state = 23)

In [ ]:
def get_metrics(y_test, predictions, binary = False):
  '''
    Esta función retorna Accuracy, Precision, Recall y F1_score.
    binary: (default: False) Booleano que indica si la clasificación es multiclase o no
  '''
  from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

  if binary == False:
    option = 'macro'
    auc = None
  if binary == True:
    option = 'binary'
    auc = roc_auc_score(y_test, predictions)
  
  acc = accuracy_score(y_test, predictions)
  precision = precision_score(y_test, predictions, average= option)
  recall = recall_score(y_test, predictions, average= option)
  f1 = f1_score(y_test, predictions, average= option)
  print('Metricas en el set de prueba...')
  print('Accuracy: ', acc)
  print('Precision: ', precision)
  print('Recall: ', recall)
  print('f1_score: ', recall)
  print('AUC: ', auc)
  print('-'*70)
  

def alg_ML(X_train, y_train, X_test, y_test, binary = False):
  from sklearn.linear_model import LogisticRegression
  from sklearn.svm import SVC
  from sklearn.tree import DecisionTreeClassifier
  from sklearn.ensemble import RandomForestClassifier

  logreg = LogisticRegression(max_iter = 10000)
  logreg.fit(X_train, y_train)
  logreg_pred = logreg.predict(X_test)
  print('Metrics for logistic regression:')
  get_metrics(y_test, logreg_pred, binary = binary)

  svm = SVC()
  svm.fit(X_train, y_train) 
  svm_pred = svm.predict(X_test)
  print('Metrics for Support Vector Machine:')
  get_metrics(y_test, svm_pred, binary = binary)

  dt = DecisionTreeClassifier()
  dt.fit(X_train, y_train)
  dt_pred = dt.predict(X_test)
  print('Metrics for Decision Tree:')
  get_metrics(y_test, dt_pred, binary = binary)

  rf = RandomForestClassifier()
  rf.fit(X_train, y_train)
  rf_pred = rf.predict(X_test)
  print('Metrics for Random Forrest: ')
  get_metrics(y_test, rf_pred, binary = binary)

In [ ]:
alg_ML(X_train, y_train, X_test, y_test, binary = True)

Metrics for logistic regression:
Metricas en el set de prueba...
Accuracy:  0.7262540021344717
Precision:  0.7259615384615384
Recall:  1.0
f1_score:  1.0
AUC:  0.5019417475728155
----------------------------------------------------------------------
Metrics for Support Vector Machine:
Metricas en el set de prueba...
Accuracy:  0.7262540021344717
Precision:  0.7262032085561497
Recall:  0.9992641648270787
f1_score:  0.9992641648270787
AUC:  0.5025447037727627
----------------------------------------------------------------------
Metrics for Decision Tree:
Metricas en el set de prueba...
Accuracy:  0.6723585912486659
Precision:  0.7807083647324793
Recall:  0.7623252391464312
f1_score:  0.7623252391464312
AUC:  0.5986383477285555
----------------------------------------------------------------------
Metrics for Random Forrest: 
Metricas en el set de prueba...
Accuracy:  0.732657417289221
Precision:  0.7910447761194029
Recall:  0.8579838116261957
f1_score:  0.8579838116261957
AUC:  0.629962

In [ ]:
#Importamos los paquetes
from imblearn.over_sampling import RandomOverSampler, SMOTE

#Inicialización de los métodos de sobremuestreo
#SMOTE
smote = SMOTE(random_state = 1)

#SMOTE. Genera nuevas muestras sintéticas
dataSmote, targetSmote = smote.fit_resample(X_train ,y_train)
Exito = targetSmote.sum()
No_exito = targetSmote.shape[0]- Exito
print('Exitos: ', Exito, ' , No_exitos: ', No_exito)

Exitos:  5441  , No_exitos:  5441


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
alg_ML(dataSmote, targetSmote, X_test, y_test, binary= True)

Metrics for logistic regression:
Metricas en el set de prueba...
Accuracy:  0.5843116328708644
Precision:  0.8014553014553014
Recall:  0.5673289183222958
f1_score:  0.5673289183222958
AUC:  0.5982275659572645
----------------------------------------------------------------------
Metrics for Support Vector Machine:
Metricas en el set de prueba...
Accuracy:  0.567769477054429
Precision:  0.8376383763837638
Recall:  0.5011037527593819
f1_score:  0.5011037527593819
AUC:  0.6223965365738657
----------------------------------------------------------------------
Metrics for Decision Tree:
Metricas en el set de prueba...
Accuracy:  0.6942369263607258
Precision:  0.7977272727272727
Recall:  0.7748344370860927
f1_score:  0.7748344370860927
AUC:  0.6281939175721726
----------------------------------------------------------------------
Metrics for Random Forrest: 
Metricas en el set de prueba...
Accuracy:  0.7193169690501601
Precision:  0.8152914458743377
Recall:  0.7924944812362031
f1_score:  0.7

## Redes Neuronales

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score

### Normalizacion MinMax
Normalizar las resenas a valores que van de 0 a 1.

In [ ]:
max = expanded['Reviews'].max()
min = expanded['Reviews'].min()
resta = max - min

expanded['Reviews'] = expanded['Reviews'].apply(lambda fila: (fila-min)/resta)
#ex = expanded['Reviews'].apply(lambda fila: (fila-media)/std)
print(expanded)

            Reviews  Tamanio_MB  ...  Categoria_VIDEO_PLAYERS  Categoria_WEATHER
0      2.021538e-06        19.0  ...                        0                  0
1      1.235953e-05        14.0  ...                        0                  0
2      1.119638e-03         8.7  ...                        0                  0
3      2.759054e-03        25.0  ...                        0                  0
4      1.235953e-05         2.8  ...                        0                  0
...             ...         ...  ...                      ...                ...
10834  7.676727e-08         2.6  ...                        0                  0
10836  4.733982e-07        53.0  ...                        0                  0
10837  3.838364e-08         3.6  ...                        0                  0
10839  1.445784e-06         7.4  ...                        0                  0
10840  5.096144e-03        19.0  ...                        0                  0

[9366 rows x 40 columns]


### Construyendo el modelo.

In [ ]:
modelo = keras.Sequential([ #Modelo mas basico de una red neuronal.
    keras.layers.Flatten(input_shape=(39,1)),        #Capa de entrada.
    keras.layers.Dense(4, activation=tf.nn.sigmoid),      #Capa oculta.
    keras.layers.Dense(1, activation=tf.nn.sigmoid)     #Capa de salida.
])

### Compilar el modelo.

In [ ]:
modelo.compile(optimizer='adam', 
               loss='binary_crossentropy', 
               metrics=['accuracy'])

### Entrenar el modelo.

In [ ]:
modelo.fit(X_train, y_train, epochs=6)

Epoch 1/6
235/235 [==============================] - 1s 1ms/step - loss: 0.5968 - accuracy: 0.7206
Epoch 2/6
235/235 [==============================] - 0s 1ms/step - loss: 0.5830 - accuracy: 0.7268
Epoch 3/6
235/235 [==============================] - 0s 1ms/step - loss: 0.5813 - accuracy: 0.7266
Epoch 4/6
235/235 [==============================] - 0s 1ms/step - loss: 0.5809 - accuracy: 0.7266
Epoch 5/6
235/235 [==============================] - 0s 1ms/step - loss: 0.5806 - accuracy: 0.7266
Epoch 6/6
235/235 [==============================] - 0s 1ms/step - loss: 0.5802 - accuracy: 0.7266


### Evaluar el modelo.

In [ ]:
test_loss, test_acc = modelo.evaluate(X_test, y_test, verbose=1)
print('Precision de las pruebas: ', test_acc)

59/59 [==============================] - 0s 1ms/step - loss: 0.5791 - accuracy: 0.7263
Precision de las pruebas:  0.7262539863586426


In [ ]:
pred = modelo.predict(X_test)
print('AUC:', roc_auc_score(y_test, pred))

AUC: 0.5646156154225337


## Maquina de soporte de vectores.

El método de sobremuestreo para la clase menos presentada en el set de prueba 'dataSmote' y 'targetSmote' mejoro levemente la métrica AUC, por lo que tomaremos este set para entrenar los modelos con SVM.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
### Parametros del model.
params = {'C': [0.01, 0.1, 1, 5, 15, 25, 50, 75, 100, 120, 150, 170, 200],
          'kernel': ['linear', 'rbf']} 
svm = SVC(max_iter= 20000)        

In [ ]:
best_svm = GridSearchCV(svm, param_grid= params, scoring= 'roc_auc', cv = 3)


In [ ]:
best_svm.fit(dataSmote, targetSmote);

In [ ]:
svm_results = pd.DataFrame(best_svm.cv_results_)
svm_results[['params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score']].to_csv('/content/drive/MyDrive/Dataset Proyecto/Bedu_Proyecto_Modulo_4/svm_results.csv')

In [ ]:
svm_results[['params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,"{'C': 0.01, 'kernel': 'linear'}",0.612487,0.637065,0.634075,0.627876,0.010949,18
1,"{'C': 0.01, 'kernel': 'rbf'}",0.600270,0.626778,0.620930,0.615993,0.011371,20
2,"{'C': 0.1, 'kernel': 'linear'}",0.636906,0.652906,0.660073,0.649962,0.009684,17
3,"{'C': 0.1, 'kernel': 'rbf'}",0.641754,0.669762,0.666733,0.659416,0.012550,13
4,"{'C': 1, 'kernel': 'linear'}",0.653533,0.661885,0.667781,0.661066,0.005845,12
5,"{'C': 1, 'kernel': 'rbf'}",0.669816,0.696608,0.700007,0.688810,0.013503,11
6,"{'C': 5, 'kernel': 'linear'}",0.655866,0.659004,0.661579,0.658817,0.002336,14
7,"{'C': 5, 'kernel': 'rbf'}",0.687784,0.704136,0.713753,0.701891,0.010720,10
8,"{'C': 15, 'kernel': 'linear'}",0.650857,0.643755,0.671427,0.655346,0.011735,15
9,"{'C': 15, 'kernel': 'rbf'}",0.700680,0.709599,0.722815,0.711032,0.009093,9


In [ ]:
pred = best_svm.predict(X_test)
roc_auc_score(y_test, pred)

0.6530865785093265